
### Keyword search and topic modelling

#### Dependencies

- `pip install beautifulsoup4 bertopic keybert keyphrase_vectorizers scikit-learn`

#### Select email folder



In [ ]:
email_folder = 'data/extracted emails/process_pst_js/alliancecoal/'

#################################

import os
email_paths = []
for root, dirs, files in os.walk(email_folder):
    for file in files:
        email_paths.append(os.path.join(root, file))

#### Keyword Extraction
- Setup

In [ ]:
import json
import random
import textwrap

from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

kw_model = KeyBERT()

def get_keywords(text, kwargs):
    if kwargs.get('vectorizer'):
        if kwargs['vectorizer'] == 'keyphrase':
            kwargs['vectorizer'] = KeyphraseCountVectorizer()
        else:
            kwargs['vectorizer'] = CountVectorizer(
                ngram_range=kwargs.get('keyphrase_ngram_range', (1, 1)),
                stop_words=kwargs['stop_words']
            )
    keywords = kw_model.extract_keywords(text, **kwargs)
    keywords = [x[0] for x in keywords]
    print('KEYWORDS:\n')
    print('\n'.join(keywords))

- Extract keywords from a random or specified email (see KWARGS)

In [ ]:
KWARGS = {
    'keyphrase_ngram_range': (1, 3), # Min, max word count for keywords
    'use_mmr': True, # Increases diversity of keywords
    'diversity': .5, # Set diversity between 0 and 1 if using MMR
    'vectorizer': 'keyphrase', # ("keyphrase", True, False) How to represent document. Keyphrase vectorizer should be more coherent
    'stop_words': 'english'
}

#################################
text = None

while not text:
    email_path = random.choice((email_paths))
    text = json.load(open(email_path, encoding='utf-8')).get('bodyText', '').strip()


print(f'FILENAME: {email_path}\n')
print('TEXT:\n ', "\n".join(textwrap.wrap(text, 100)), '\n')
get_keywords(text, KWARGS)

# Run on same email with different args
# print()
# KWARGS['vectorizer'] = True
# get_keywords(email, KWARGS)

text = None

#### Topic Modeling

- Imports

In [ ]:
import json
import random
import textwrap

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

- Get topics and view topic hierarchy (see comments)

In [ ]:
# Select subset or all emails in email folder
slice_docs = (None, None)

# Document representations to chain and feed into topic model
representations = [
    KeyBERTInspired(), # Should make topics more coherent
    MaximalMarginalRelevance(diversity=0.3), # Makes topics more diverse
]

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

topic_model = BERTopic(
        representation_model=representations, # Representations (see above)
        ctfidf_model=ctfidf_model,  # Prevents very frequent words in data from being candidate topics
        nr_topics='auto' # Topic reduction. Set number of desired topics, 'auto' for auto-reduction, 
        # or None. Set to None if there aren't enough topic. 
        )

#################################

docs = [json.load(open(path, encoding='utf-8')).get('bodyText', '') for 
        path in email_paths[slice_docs[0]:slice_docs[1]]]
docs = [doc for doc in docs if doc.strip()]

topics, probabilities = topic_model.fit_transform(docs)
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

- View topics as table

In [ ]:
topic_model.get_topic_info()

- Show docs per topic(s)

In [ ]:
# Select topic(s) by number (must be list). Get topic numbers from table or 
# labels in topic tree , ie. "1_trees_grass_nature" topic number is 1. 
topics_to_show = [0]
# Show first n docs
n_docs_to_show = 10
# Show first n characters of each doc
n_chars_per_doc = 500

#################################

df = pd.DataFrame({"Document": docs, "Topic": topics})
df = df.loc[df['Topic'].isin(topics_to_show)].head(n_docs_to_show)
docs_ = df['Document'].to_list()
random.shuffle(docs_)
for i, doc in enumerate(docs_):
    print(f'DOC {i + 1}\n----------\n')
    print('\n'.join(textwrap.wrap(doc[:n_chars_per_doc] + ' ...' if n_chars_per_doc else '')), '\n')